In [1]:
import numpy as np
import matplotlib.pyplot as plt
import IPython
from scipy.io import wavfile
%matplotlib inline

In [2]:
fs, s = wavfile.read('../samples/cmu_arctic_us_aew_a0002.wav')
s = s/np.max(s)

In [3]:
# Read input signal
IPython.display.Audio(s,rate=fs)

In [4]:
def compute_parameters(s,p):
    """Compute the parameters a_k of the p-order all-pole filter by solving the linear system obtained by MAP with
    observed signal s"""
    N = len(s)
    A = np.zeros((p,p))
    b = np.zeros(p)
    
    # For now I'm doing stuff with for loops which is very inefficient, could probably be improved by
    # 1. Considering the symmetries of matrix A
    # 2. Using smart numpy stuff
    for i in range(1,p+1):
        b[i-1] = s[p:N]@s[p-i:N-i] # Construct RHS
        for j in range(1,p+1):
            A[i-1,j-1] = s[p-i:N-i]@s[p-j:N-j] #Construct LHS
    a = np.linalg.solve(A,b)
    return a        

In [5]:
p = 20 # Order of the filter
frame_size = 400 
nframes = int(len(s)/frame_size) # A bit rough
y = np.zeros(len(s)) # Synthetised signal
u = np.random.randn(len(s)) # Excitation signal, right now : for unvoiced speech. Should add frequency detector for voiced pitch

for frame in range(1,nframes): # Right now : not filling the first frame. Should pad y with p samples at the beginning
    start = frame*frame_size
    end = ((frame+1)*frame_size)
    a = compute_parameters(s[start:end],p)
    for n in range(start,end):
        y[n] = a@np.flip(y[n-p:n],0) + u[n] # Synthetize signal using the filter + the excitation

y = y/np.max(y)

In [7]:
#Read synthetised signal
IPython.display.Audio(y,rate=fs)